In [48]:
import numpy as np
import xarray as xr
import os
import glob
from pathlib import Path 
import pathlib
from pprint import pprint
import matplotlib.pyplot as plt
import time
import requests
import s3fs
from os.path import dirname, join
import datetime
import myDate
import netCDF4

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import warnings
warnings.filterwarnings('ignore')

In [49]:
# alongtrack_file_dir = Path('/home/jpluser/ECCO_GMSL/Data/ECCO_V4r4_alongtrack_output_nosicapplied/')
# alongtrack_file_dir_randomnoise = Path('/home/jpluser/ECCO_GMSL/Data/ECCO_V4r4_alongtrack_output_randomnoise_nosicapplied/')
# alongtrack_file_dir_missingdata = Path('/home/jpluser/ECCO_GMSL/Data/ECCO_V4r4_alongtrack_output_missingdata_nosicapplied/')
# alongtrack_file_dir_orbiterror_2cm = Path('/home/jpluser/ECCO_GMSL/Data/ECCO_V4r4_alongtrack_output_orbiterror_2cm_nosicapplied/')
# alongtrack_file_dir_3errors_2cm = Path('/home/jpluser/ECCO_GMSL/Data/ECCO_V4r4_alongtrack_output_3errors_2cm_nosicapplied/')
grids_file_dir = Path('/home/jpluser/ECCO_GMSL/Data/gridding_output_nosicapplied/')
grids_file_dir_randomnoise = Path('/home/jpluser/ECCO_GMSL/Data/gridding_output_randomnoise_nosicapplied/')
grids_file_dir_missingdata = Path('/home/jpluser/ECCO_GMSL/Data/gridding_output_missingdata_nosicapplied/')
grids_file_dir_orbiterror_2cm = Path('/home/jpluser/ECCO_GMSL/Data/gridding_output_orbiterror_2cm_nosicapplied/')
grids_file_dir_3errors_2cm = Path('/home/jpluser/ECCO_GMSL/Data/gridding_output_3errors_2cm_nosicapplied/')
grids_file_dir_beckley = Path('/home/jpluser/ECCO_GMSL/Data/gridding_output_Beckley_dailyfiles/')

main_dir = Path('/home/jpluser/ECCO_GMSL/')
output_dir = Path('/home/jpluser/ECCO_GMSL/Data/GMSL/')
fig_dir = Path('/home/jpluser/ECCO_GMSL/Figures/')

# Compute GMSL from simple gridder of Beckley along track data daily files

In [3]:
beckley_grids_files = list(grids_file_dir_beckley.glob('ssha_global_half_deg_*nc'))
beckley_grids_files=np.sort(beckley_grids_files)
print(beckley_grids_files[0:5])
print('...')
print(beckley_grids_files[-5:])

[PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_Beckley_dailyfiles/ssha_global_half_deg_19920927.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_Beckley_dailyfiles/ssha_global_half_deg_19921007.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_Beckley_dailyfiles/ssha_global_half_deg_19921017.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_Beckley_dailyfiles/ssha_global_half_deg_19921027.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_Beckley_dailyfiles/ssha_global_half_deg_19921106.nc')]
...
[PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_Beckley_dailyfiles/ssha_global_half_deg_20171125.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_Beckley_dailyfiles/ssha_global_half_deg_20171205.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_Beckley_dailyfiles/ssha_global_half_deg_20171215.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_Beckley_dailyfiles/ssha_global_half_deg_20171225.n

In [4]:
grids = xr.open_mfdataset(
        paths=beckley_grids_files,
        combine='nested',
        concat_dim='time',
        decode_cf=True,
        compat='override',
        coords='minimal',
        chunks={'time': 1}  
    )

In [5]:
grids=grids.sel(latitude=slice(-66,66)).sel(time=slice('1992-09-23','2017-12-31'))
grids

<xarray.Dataset> Size: 3GB
Dimensions:    (time: 922, latitude: 264, longitude: 720)
Coordinates:
  * latitude   (latitude) float64 2kB -65.75 -65.25 -64.75 ... 64.75 65.25 65.75
  * longitude  (longitude) float64 6kB -179.8 -179.2 -178.8 ... 179.2 179.8
  * time       (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2017-12-25
Data variables:
    SSHA       (time, latitude, longitude) float64 1GB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float64 1GB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>

In [6]:
[LO,LA]=np.meshgrid(grids.longitude,grids.latitude)
weights = np.cos(np.deg2rad(LA))

In [7]:
weights=np.tile(weights, [grids.SSHA.shape[0], 1, 1])
weights[np.where(~np.isfinite(grids.SSHA))]=np.nan
weights.shape

(922, 264, 720)

In [8]:
ssh_weighted_mean=np.nansum(grids.SSHA*weights,axis=(1,2))/np.nansum(weights,axis=(1,2))

In [9]:
# make new DataArray objects
gmsl_grids_beckley = xr.DataArray(ssh_weighted_mean, dims=['time'])
gmsl_grids_beckley = gmsl_grids_beckley.assign_coords({'time':grids.time.data})
gmsl_grids_beckley.name = 'gmsl_grids_beckley'
gmsl_grids_beckley

<xarray.DataArray 'gmsl_grids_beckley' (time: 922)> Size: 7kB
array([ 1.55574287e-02,  1.23287737e-02,  2.20423251e-03, -3.73340125e-03,
       -5.19729889e-03, -8.97283127e-03, -2.19258951e-03, -1.37798626e-02,
       -1.62856912e-02, -1.86743812e-02, -1.98397839e-02, -2.24182594e-02,
       -2.40730534e-02, -2.21298541e-02, -2.00285762e-02, -1.88103125e-02,
       -1.94303392e-02, -1.85841789e-02, -2.06701992e-02, -2.18809600e-02,
       -2.05283399e-02, -1.89631396e-02, -2.14559979e-02, -2.22356334e-02,
       -2.35134478e-02, -2.36238705e-02, -2.11505628e-02, -2.01487745e-02,
       -1.93981352e-02, -1.83678927e-02, -1.47626550e-02, -1.84043314e-02,
       -1.59006245e-02, -1.27668234e-02, -1.26823000e-02, -1.39823931e-02,
       -1.38957327e-02, -1.00234353e-02, -5.83329084e-03, -9.48767321e-03,
       -1.26449497e-02, -1.30323976e-02, -1.38137990e-02, -1.46507135e-02,
       -1.11854304e-02, -9.77368543e-03, -1.15972843e-02, -1.38501240e-02,
       -1.07460201e-02, -1.28332678e-02, -1.97316384e-02, -1.98138940e-02,
       -1.72499928e-02, -1.63739355e-02, -1.43135072e-02, -1.36788457e-02,
       -1.47647801e-02, -1.58990659e-02, -1.81852706e-02, -1.87201343e-02,
       -1.99810404e-02, -1.94262403e-02, -1.89754378e-02, -1.88470677e-02,
       -2.39769768e-02, -1.72726234e-02, -1.62210227e-02, -1.58995981e-02,
       -1.46499931e-02, -1.28709826e-02, -1.13150953e-02, -8.64645784e-03,
       -8.06396879e-03, -8.80989807e-03, -6.89097886e-03, -5.62926884e-03,
       -5.69166239e-03, -1.39904774e-03, -6.44951839e-03, -1.06606147e-02,
...
        5.94980670e-02,  6.12418196e-02,  6.06442166e-02,  5.78896055e-02,
        5.64423126e-02,  5.32181017e-02,  5.82797050e-02,  5.93661594e-02,
        5.54234949e-02,  5.76859523e-02,  5.51881905e-02,  5.55995541e-02,
        5.32159062e-02,  5.68509288e-02,  5.04687345e-02,  5.18218472e-02,
        5.01768530e-02,  4.87811784e-02,  4.82623748e-02,  4.78809487e-02,
        5.11989096e-02,  5.21698337e-02,  5.31333366e-02,  5.28191258e-02,
        5.24863049e-02,  5.12160584e-02,  5.24019711e-02,  5.21164553e-02,
        5.32739960e-02,  5.97683418e-02,  6.17365252e-02,  6.06095228e-02,
        5.84041858e-02,  6.02212538e-02,  6.37192558e-02,  6.29939154e-02,
        6.43674117e-02,  6.54724832e-02,  6.35231036e-02,  6.01112442e-02,
        5.95638205e-02,  5.72129384e-02,  5.68564719e-02,  5.52069416e-02,
        5.34305668e-02,  5.09547812e-02,  5.34471104e-02,  5.28860429e-02,
        5.33855770e-02,  5.44034011e-02,  5.30854220e-02,  5.19655207e-02,
        5.24116034e-02,  5.05366155e-02,  4.88301095e-02,  4.89225638e-02,
        4.91645877e-02,  5.07703620e-02,  5.14549647e-02,  5.17119769e-02,
        5.24624130e-02,  5.19290906e-02,  5.69342052e-02,  5.78342237e-02,
        5.96996641e-02,  6.06688648e-02,  6.30898334e-02,  6.38086690e-02,
        6.29748308e-02,  6.43986253e-02,  6.55300682e-02,  6.67102696e-02,
        6.60872760e-02,  6.69667992e-02,  6.73075971e-02,  6.59661136e-02,
        6.53915225e-02,  6.20537112e-02])
Coordinates:
  * time     (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2017-12-25

In [10]:
fname = output_dir / ('GMSL_Beckley_dailyfiles_grids.nc')
gmsl_grids_beckley.to_netcdf(fname)

# Compute GMSL from ECCO synthetic grids with simple weighing

### No error

In [11]:
# After running Kevin's code simple gridder.py, grids are generating
ECCO_grids_files = list(grids_file_dir.glob('*SSHA_gridded_*nc'))
ECCO_grids_files=np.sort(ECCO_grids_files)
print(ECCO_grids_files[0:5])
print('...')
print(ECCO_grids_files[-5:])

[PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_nosicapplied/SSHA_gridded_1992-09-27.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_nosicapplied/SSHA_gridded_1992-10-07.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_nosicapplied/SSHA_gridded_1992-10-17.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_nosicapplied/SSHA_gridded_1992-10-27.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_nosicapplied/SSHA_gridded_1992-11-06.nc')]
...
[PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_nosicapplied/SSHA_gridded_2017-11-25.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_nosicapplied/SSHA_gridded_2017-12-05.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_nosicapplied/SSHA_gridded_2017-12-15.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_nosicapplied/SSHA_gridded_2017-12-25.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_nosicapplied/SSHA_gridded_2018-01-04.nc')]


In [12]:
grids = xr.open_mfdataset(
        paths=ECCO_grids_files,
        combine='nested',
        concat_dim='time',
        decode_cf=True,
        compat='override',
        coords='minimal',
        chunks={'time': 1}  
    )   
grids

<xarray.Dataset> Size: 4GB
Dimensions:    (time: 924, latitude: 360, longitude: 720)
Coordinates:
  * latitude   (latitude) float64 3kB -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * longitude  (longitude) float64 6kB -179.8 -179.2 -178.8 ... 179.2 179.8
  * time       (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float64 2GB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float64 2GB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>

In [13]:
grids=grids.where(np.array(grids.counts)>=100)

In [14]:
grids=grids.sel(latitude=slice(-66,66)).sel(time=slice('1992-09-23','2017-12-31'))
grids

<xarray.Dataset> Size: 3GB
Dimensions:    (time: 923, latitude: 264, longitude: 720)
Coordinates:
  * latitude   (latitude) float64 2kB -65.75 -65.25 -64.75 ... 64.75 65.25 65.75
  * longitude  (longitude) float64 6kB -179.8 -179.2 -178.8 ... 179.2 179.8
  * time       (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2017-12-25
Data variables:
    SSHA       (time, latitude, longitude) float64 1GB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float64 1GB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>

In [15]:
[LO,LA]=np.meshgrid(grids.longitude,grids.latitude)
weights = np.cos(np.deg2rad(LA))
weights=np.tile(weights, [grids.SSHA.shape[0], 1, 1])
weights[np.where(~np.isfinite(grids.SSHA))]=np.nan
ssh_weighted_mean=np.nansum(grids.SSHA*weights,axis=(1,2))/np.nansum(weights,axis=(1,2))

In [16]:
# make new DataArray objects
gmsl_grids = xr.DataArray(ssh_weighted_mean, dims=['time'])
gmsl_grids = gmsl_grids.assign_coords({'time':grids.time})
gmsl_grids.name = 'gmsl_grids'
gmsl_grids

<xarray.DataArray 'gmsl_grids' (time: 923)> Size: 7kB
array([-3.36603441e-02, -3.15579380e-02, -2.94811656e-02, -3.00407671e-02,
       -3.11933572e-02, -3.29836895e-02, -3.12716216e-02, -3.27510644e-02,
       -3.31704040e-02, -3.88971745e-02, -4.08745537e-02, -4.04771779e-02,
       -4.09706026e-02, -4.25091364e-02, -4.28531172e-02, -4.01991540e-02,
       -3.99531709e-02, -3.84186812e-02, -4.09582185e-02, -3.88633222e-02,
       -3.80041866e-02, -3.73317158e-02, -3.87022750e-02, -3.86248244e-02,
       -3.72427423e-02, -3.70629628e-02, -3.80155615e-02, -3.76047539e-02,
       -3.92606809e-02, -3.79095040e-02, -3.67494460e-02, -3.74287591e-02,
       -3.47359870e-02, -3.39323439e-02, -3.23607863e-02, -3.21639476e-02,
       -3.06438555e-02, -2.84482750e-02, -2.76684725e-02, -2.78042698e-02,
       -2.93610539e-02, -3.01101752e-02, -3.26672921e-02, -3.19110857e-02,
       -2.84233810e-02, -2.69010899e-02, -3.16000835e-02, -3.24417510e-02,
       -3.06499634e-02, -3.42212722e-02, -3.83829670e-02, -3.93126618e-02,
       -3.51347780e-02, -3.33243037e-02, -3.22077095e-02, -3.34223625e-02,
       -3.18155737e-02, -3.34038605e-02, -3.33402921e-02, -3.64072192e-02,
       -3.60423739e-02, -3.57248767e-02, -3.87860219e-02, -3.95929178e-02,
       -4.02243814e-02, -3.92274487e-02, -3.75311218e-02, -3.71165454e-02,
       -3.59460734e-02, -3.57409874e-02, -3.21554462e-02, -3.24485229e-02,
       -2.99541344e-02, -2.97680394e-02, -2.69432956e-02, -2.77066323e-02,
       -2.54290450e-02, -2.63693809e-02, -2.78497772e-02, -2.83337865e-02,
...
        4.68583794e-02,  4.60499114e-02,  4.74178157e-02,  4.49212303e-02,
        4.37815207e-02,  3.97807841e-02,  3.83239280e-02,  4.19627928e-02,
        4.11805828e-02,  3.77620229e-02,  3.81375864e-02,  3.64807992e-02,
        3.81886171e-02,  3.57997029e-02,  3.86025867e-02,  3.64187828e-02,
        3.75090242e-02,  3.70630420e-02,  3.90407266e-02,  3.84797571e-02,
        3.65472181e-02,  3.80557088e-02,  3.72833228e-02,  3.69073865e-02,
        3.70418016e-02,  3.64095062e-02,  3.52794938e-02,  3.85974195e-02,
        3.90595517e-02,  3.98589714e-02,  4.18528173e-02,  4.32302421e-02,
        4.26228663e-02,  4.25377011e-02,  4.40344534e-02,  4.66523322e-02,
        4.63702471e-02,  4.84890328e-02,  4.76875797e-02,  4.90507590e-02,
        4.71900851e-02,  4.68822523e-02,  4.51544283e-02,  4.52508492e-02,
        4.25090754e-02,  3.87949852e-02,  3.69987773e-02,  3.75608817e-02,
        3.72742420e-02,  3.77299490e-02,  3.75843008e-02,  3.77041435e-02,
        3.94172349e-02,  4.04393334e-02,  4.11079007e-02,  4.16350970e-02,
        4.00017064e-02,  3.98996139e-02,  4.25784031e-02,  4.12260669e-02,
        4.17803936e-02,  4.03083005e-02,  4.09401959e-02,  4.32330536e-02,
        4.41135692e-02,  4.65887745e-02,  4.70464333e-02,  4.77432192e-02,
        4.96219788e-02,  4.75283323e-02,  4.88262840e-02,  4.94728311e-02,
        5.26132463e-02,  5.39287144e-02,  5.44362756e-02,  5.66998964e-02,
        5.50421639e-02,  5.58879955e-02,  5.38383615e-02])
Coordinates:
  * time     (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2017-12-25

### Random noise

In [17]:
# After running Kevin's code simple gridder.py, grids are generating
ECCO_grids_files = list(grids_file_dir_randomnoise.glob('*SSHA_gridded_*nc'))
ECCO_grids_files=np.sort(ECCO_grids_files)
print(ECCO_grids_files[0:5])
print('...')
print(ECCO_grids_files[-5:])

[PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_randomnoise_nosicapplied/SSHA_gridded_1992-09-27.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_randomnoise_nosicapplied/SSHA_gridded_1992-10-07.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_randomnoise_nosicapplied/SSHA_gridded_1992-10-17.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_randomnoise_nosicapplied/SSHA_gridded_1992-10-27.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_randomnoise_nosicapplied/SSHA_gridded_1992-11-06.nc')]
...
[PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_randomnoise_nosicapplied/SSHA_gridded_2017-11-25.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_randomnoise_nosicapplied/SSHA_gridded_2017-12-05.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_randomnoise_nosicapplied/SSHA_gridded_2017-12-15.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_randomnoise_nosicapplied/SSHA_gridded_2017-12-25.n

In [18]:
grids = xr.open_mfdataset(
        paths=ECCO_grids_files,
        combine='nested',
        concat_dim='time',
        decode_cf=True,
        compat='override',
        coords='minimal',
        chunks={'time': 1}  
    )   
grids

<xarray.Dataset> Size: 4GB
Dimensions:    (time: 924, latitude: 360, longitude: 720)
Coordinates:
  * latitude   (latitude) float64 3kB -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * longitude  (longitude) float64 6kB -179.8 -179.2 -178.8 ... 179.2 179.8
  * time       (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float64 2GB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float64 2GB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>

In [19]:
grids=grids.where(np.array(grids.counts)>=100)
grids=grids.sel(latitude=slice(-66,66)).sel(time=slice('1992-09-23','2017-12-31'))
grids

<xarray.Dataset> Size: 3GB
Dimensions:    (time: 923, latitude: 264, longitude: 720)
Coordinates:
  * latitude   (latitude) float64 2kB -65.75 -65.25 -64.75 ... 64.75 65.25 65.75
  * longitude  (longitude) float64 6kB -179.8 -179.2 -178.8 ... 179.2 179.8
  * time       (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2017-12-25
Data variables:
    SSHA       (time, latitude, longitude) float64 1GB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float64 1GB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>

In [20]:
# weights = np.cos(np.deg2rad(grids.latitude))
# weights.name = "weights"
# weights

In [21]:
# ssh_weighted = grids.weighted(weights,skipna=True)
# ssh_weighted

In [22]:
# ssh_weighted_mean_randomnoise = ssh_weighted.mean(("longitude", "latitude"),skipna=True)
# ssh_weighted_mean_randomnoise.SSHA

In [23]:
[LO,LA]=np.meshgrid(grids.longitude,grids.latitude)
weights = np.cos(np.deg2rad(LA))
weights=np.tile(weights, [grids.SSHA.shape[0], 1, 1])
weights[np.where(~np.isfinite(grids.SSHA))]=np.nan
ssh_weighted_mean_randomnoise=np.nansum(grids.SSHA*weights,axis=(1,2))/np.nansum(weights,axis=(1,2))

In [24]:
# make new DataArray objects
gmsl_grids_randomnoise = xr.DataArray(ssh_weighted_mean_randomnoise, dims=['time'])
gmsl_grids_randomnoise = gmsl_grids_randomnoise.assign_coords({'time':grids.time})
gmsl_grids_randomnoise.name = 'gmsl_grids'
gmsl_grids_randomnoise

<xarray.DataArray 'gmsl_grids' (time: 923)> Size: 7kB
array([-3.36829565e-02, -3.15353211e-02, -2.94468340e-02, -3.00572198e-02,
       -3.11597019e-02, -3.29955222e-02, -3.12856059e-02, -3.27471919e-02,
       -3.30951387e-02, -3.89106527e-02, -4.08929872e-02, -4.04573441e-02,
       -4.09649606e-02, -4.25202935e-02, -4.27944270e-02, -4.01538833e-02,
       -3.99221115e-02, -3.83889628e-02, -4.09928850e-02, -3.88366901e-02,
       -3.79761930e-02, -3.73233350e-02, -3.86961761e-02, -3.85608393e-02,
       -3.71839445e-02, -3.70844788e-02, -3.78965093e-02, -3.76147857e-02,
       -3.92650542e-02, -3.78694155e-02, -3.67299774e-02, -3.74309892e-02,
       -3.47830697e-02, -3.39745823e-02, -3.23490659e-02, -3.21782345e-02,
       -3.06981298e-02, -2.84692810e-02, -2.76245406e-02, -2.77555943e-02,
       -2.93470661e-02, -3.00786019e-02, -3.26173502e-02, -3.19534867e-02,
       -2.83582231e-02, -2.68917334e-02, -3.16073945e-02, -3.24017486e-02,
       -3.06719981e-02, -3.41697537e-02, -3.83780507e-02, -3.93510034e-02,
       -3.51784103e-02, -3.32177003e-02, -3.22427209e-02, -3.34224702e-02,
       -3.18643951e-02, -3.34839147e-02, -3.33627678e-02, -3.64004839e-02,
       -3.60455964e-02, -3.57464437e-02, -3.87484429e-02, -3.96314904e-02,
       -4.02034062e-02, -3.91391959e-02, -3.75502542e-02, -3.71738020e-02,
       -3.59504786e-02, -3.57454993e-02, -3.21145997e-02, -3.25053359e-02,
       -2.99784523e-02, -2.97367972e-02, -2.68985246e-02, -2.77735215e-02,
       -2.54868131e-02, -2.62914013e-02, -2.78891324e-02, -2.83344777e-02,
...
        4.68749987e-02,  4.60297770e-02,  4.73689991e-02,  4.49368533e-02,
        4.37414781e-02,  3.98293914e-02,  3.82586109e-02,  4.19425787e-02,
        4.11738800e-02,  3.77782480e-02,  3.81135920e-02,  3.64858312e-02,
        3.81861168e-02,  3.58159451e-02,  3.84919780e-02,  3.63555670e-02,
        3.76056586e-02,  3.70291795e-02,  3.91140115e-02,  3.84828665e-02,
        3.65597973e-02,  3.81313069e-02,  3.72978405e-02,  3.69389616e-02,
        3.70666223e-02,  3.64190360e-02,  3.52271153e-02,  3.86390976e-02,
        3.90969576e-02,  3.99085551e-02,  4.17916878e-02,  4.31525795e-02,
        4.26538550e-02,  4.25213107e-02,  4.39568106e-02,  4.66734492e-02,
        4.63994835e-02,  4.85129299e-02,  4.77237038e-02,  4.90976719e-02,
        4.71393246e-02,  4.68441866e-02,  4.51954628e-02,  4.52873139e-02,
        4.25265201e-02,  3.87464420e-02,  3.69801039e-02,  3.76151611e-02,
        3.72761826e-02,  3.76693130e-02,  3.76467986e-02,  3.76404279e-02,
        3.94318067e-02,  4.04808392e-02,  4.10597091e-02,  4.16931608e-02,
        3.99744622e-02,  3.99125126e-02,  4.26111096e-02,  4.12180264e-02,
        4.17859511e-02,  4.03240621e-02,  4.09547946e-02,  4.32318473e-02,
        4.40219490e-02,  4.65989032e-02,  4.70266191e-02,  4.77781099e-02,
        4.96727001e-02,  4.75441601e-02,  4.88235451e-02,  4.93864454e-02,
        5.25534752e-02,  5.39386961e-02,  5.44294048e-02,  5.67747592e-02,
        5.50355071e-02,  5.57748378e-02,  5.38275534e-02])
Coordinates:
  * time     (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2017-12-25

### Missing data

In [25]:
# After running Kevin's code simple gridder.py, grids are generating
ECCO_grids_files = list(grids_file_dir_missingdata.glob('*SSHA_gridded_*nc'))
ECCO_grids_files=np.sort(ECCO_grids_files)
print(ECCO_grids_files[0:5])
print('...')
print(ECCO_grids_files[-5:])

[PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_missingdata_nosicapplied/SSHA_gridded_1992-09-27.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_missingdata_nosicapplied/SSHA_gridded_1992-10-07.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_missingdata_nosicapplied/SSHA_gridded_1992-10-17.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_missingdata_nosicapplied/SSHA_gridded_1992-10-27.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_missingdata_nosicapplied/SSHA_gridded_1992-11-06.nc')]
...
[PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_missingdata_nosicapplied/SSHA_gridded_2017-11-25.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_missingdata_nosicapplied/SSHA_gridded_2017-12-05.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_missingdata_nosicapplied/SSHA_gridded_2017-12-15.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_missingdata_nosicapplied/SSHA_gridded_2017-12-25.n

In [26]:
grids = xr.open_mfdataset(
        paths=ECCO_grids_files,
        combine='nested',
        concat_dim='time',
        decode_cf=True,
        compat='override',
        coords='minimal',
        chunks={'time': 1}  
    )   
grids

<xarray.Dataset> Size: 4GB
Dimensions:    (time: 924, latitude: 360, longitude: 720)
Coordinates:
  * latitude   (latitude) float64 3kB -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * longitude  (longitude) float64 6kB -179.8 -179.2 -178.8 ... 179.2 179.8
  * time       (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float64 2GB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float64 2GB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>

In [27]:
grids=grids.where(np.array(grids.counts)>=100)
grids=grids.sel(latitude=slice(-66,66)).sel(time=slice('1992-09-23','2017-12-31'))
grids

<xarray.Dataset> Size: 3GB
Dimensions:    (time: 923, latitude: 264, longitude: 720)
Coordinates:
  * latitude   (latitude) float64 2kB -65.75 -65.25 -64.75 ... 64.75 65.25 65.75
  * longitude  (longitude) float64 6kB -179.8 -179.2 -178.8 ... 179.2 179.8
  * time       (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2017-12-25
Data variables:
    SSHA       (time, latitude, longitude) float64 1GB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float64 1GB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>

In [28]:
# weights = np.cos(np.deg2rad(grids.latitude))
# weights.name = "weights"
# weights

In [29]:
# ssh_weighted = grids.weighted(weights,skipna=True)
# ssh_weighted

In [30]:
# ssh_weighted_mean_missingdata = ssh_weighted.mean(("longitude", "latitude"),skipna=True)
# ssh_weighted_mean_missingdata.SSHA

In [31]:
[LO,LA]=np.meshgrid(grids.longitude,grids.latitude)
weights = np.cos(np.deg2rad(LA))
weights=np.tile(weights, [grids.SSHA.shape[0], 1, 1])
weights[np.where(~np.isfinite(grids.SSHA))]=np.nan
ssh_weighted_mean_missingdata=np.nansum(grids.SSHA*weights,axis=(1,2))/np.nansum(weights,axis=(1,2))

In [32]:
# make new DataArray objects
gmsl_grids_missingdata = xr.DataArray(ssh_weighted_mean_missingdata, dims=['time'])
gmsl_grids_missingdata = gmsl_grids_missingdata.assign_coords({'time':grids.time})
gmsl_grids_missingdata.name = 'gmsl_grids'
gmsl_grids_missingdata

<xarray.DataArray 'gmsl_grids' (time: 923)> Size: 7kB
array([-3.36617861e-02, -3.16151949e-02, -2.99914569e-02, -3.01451888e-02,
       -3.13040486e-02, -3.34223006e-02, -3.14635801e-02, -3.29266995e-02,
       -3.28411985e-02, -3.87351071e-02, -4.04317017e-02, -4.00769421e-02,
       -4.05639775e-02, -4.19442476e-02, -4.25913626e-02, -3.98601853e-02,
       -3.97919078e-02, -3.78287972e-02, -4.06955228e-02, -3.85971198e-02,
       -3.78502675e-02, -3.71837804e-02, -3.83759046e-02, -3.86154192e-02,
       -3.72913332e-02, -3.68173911e-02, -3.80671429e-02, -3.74244385e-02,
       -3.92651899e-02, -3.78117688e-02, -3.58379800e-02, -3.74811044e-02,
       -3.47773126e-02, -3.37005634e-02, -3.23693197e-02, -3.22052743e-02,
       -3.05230038e-02, -2.84284265e-02, -2.74170904e-02, -2.77722689e-02,
       -2.95119691e-02, -3.01751510e-02, -3.31258059e-02, -3.15678836e-02,
       -2.84462575e-02, -2.66859589e-02, -3.13014514e-02, -3.23645739e-02,
       -3.01941902e-02, -3.37513433e-02, -3.82115208e-02, -3.94864793e-02,
       -3.48432056e-02, -3.28901510e-02, -3.21480759e-02, -3.30039330e-02,
       -3.15165346e-02, -3.33094629e-02, -3.29951451e-02, -3.64086631e-02,
       -3.59861443e-02, -3.56374201e-02, -3.85743528e-02, -3.95263977e-02,
       -4.06119779e-02, -3.91854188e-02, -3.76756055e-02, -3.69670103e-02,
       -3.59311027e-02, -3.59576375e-02, -3.23334251e-02, -3.22421978e-02,
       -2.96425519e-02, -3.02429795e-02, -2.68360402e-02, -2.76323709e-02,
       -2.56721752e-02, -2.62926712e-02, -2.88263645e-02, -2.84678928e-02,
...
        4.68892350e-02,  4.61312426e-02,  4.75202599e-02,  4.48367303e-02,
        4.35102408e-02,  3.95664507e-02,  3.81288261e-02,  4.19717648e-02,
        4.08655977e-02,  3.75869960e-02,  3.81381791e-02,  3.67311603e-02,
        3.85651101e-02,  3.58213229e-02,  3.87888922e-02,  3.63591413e-02,
        3.76965958e-02,  3.71980754e-02,  3.92365892e-02,  3.84205717e-02,
        3.67383723e-02,  3.80402903e-02,  3.73565855e-02,  3.69628697e-02,
        3.70331461e-02,  3.65445996e-02,  3.52478171e-02,  3.84963679e-02,
        3.89771086e-02,  3.98371663e-02,  4.20465616e-02,  4.35950519e-02,
        4.25569848e-02,  4.26326983e-02,  4.41658686e-02,  4.68088770e-02,
        4.65770892e-02,  4.84191725e-02,  4.76890452e-02,  4.89812142e-02,
        4.70038927e-02,  4.66906388e-02,  4.50375488e-02,  4.51002567e-02,
        4.21912322e-02,  3.87389363e-02,  3.69942625e-02,  3.78300094e-02,
        3.72557109e-02,  3.79055043e-02,  3.78623571e-02,  3.79214512e-02,
        3.94056842e-02,  4.05930780e-02,  4.14237266e-02,  4.16130173e-02,
        4.00459804e-02,  3.98424832e-02,  4.25701091e-02,  4.11762943e-02,
        4.15206384e-02,  4.02120999e-02,  4.09715016e-02,  4.32111735e-02,
        4.39002358e-02,  4.65632049e-02,  4.69637203e-02,  4.77075673e-02,
        4.97587793e-02,  4.74921885e-02,  4.88239180e-02,  4.95403845e-02,
        5.27014448e-02,  5.39861871e-02,  5.42641084e-02,  5.65774058e-02,
        5.49074849e-02,  5.58115996e-02,  5.36698313e-02])
Coordinates:
  * time     (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2017-12-25

### Orbit Error - 2cm

In [33]:
# After running Kevin's code simple gridder.py, grids are generating
ECCO_grids_files = list(grids_file_dir_orbiterror_2cm.glob('*SSHA_gridded_*nc'))
ECCO_grids_files=np.sort(ECCO_grids_files)
print(ECCO_grids_files[0:5])
print('...')
print(ECCO_grids_files[-5:])

[PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_orbiterror_2cm_nosicapplied/SSHA_gridded_1992-09-27.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_orbiterror_2cm_nosicapplied/SSHA_gridded_1992-10-07.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_orbiterror_2cm_nosicapplied/SSHA_gridded_1992-10-17.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_orbiterror_2cm_nosicapplied/SSHA_gridded_1992-10-27.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_orbiterror_2cm_nosicapplied/SSHA_gridded_1992-11-06.nc')]
...
[PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_orbiterror_2cm_nosicapplied/SSHA_gridded_2017-11-25.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_orbiterror_2cm_nosicapplied/SSHA_gridded_2017-12-05.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_orbiterror_2cm_nosicapplied/SSHA_gridded_2017-12-15.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_orbiterror_2cm_nosicapplie

In [34]:
grids = xr.open_mfdataset(
        paths=ECCO_grids_files,
        combine='nested',
        concat_dim='time',
        decode_cf=True,
        compat='override',
        coords='minimal',
        chunks={'time': 1}  
    )   
grids

<xarray.Dataset> Size: 4GB
Dimensions:    (time: 924, latitude: 360, longitude: 720)
Coordinates:
  * latitude   (latitude) float64 3kB -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * longitude  (longitude) float64 6kB -179.8 -179.2 -178.8 ... 179.2 179.8
  * time       (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float64 2GB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float64 2GB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>

In [35]:
grids=grids.where(np.array(grids.counts)>=100)
grids=grids.sel(latitude=slice(-66,66)).sel(time=slice('1992-09-23','2017-12-31'))
grids

<xarray.Dataset> Size: 3GB
Dimensions:    (time: 923, latitude: 264, longitude: 720)
Coordinates:
  * latitude   (latitude) float64 2kB -65.75 -65.25 -64.75 ... 64.75 65.25 65.75
  * longitude  (longitude) float64 6kB -179.8 -179.2 -178.8 ... 179.2 179.8
  * time       (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2017-12-25
Data variables:
    SSHA       (time, latitude, longitude) float64 1GB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float64 1GB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>

In [36]:
[LO,LA]=np.meshgrid(grids.longitude,grids.latitude)
weights = np.cos(np.deg2rad(LA))
weights=np.tile(weights, [grids.SSHA.shape[0], 1, 1])
weights[np.where(~np.isfinite(grids.SSHA))]=np.nan
ssh_weighted_mean_orbiterror=np.nansum(grids.SSHA*weights,axis=(1,2))/np.nansum(weights,axis=(1,2))

In [37]:
# make new DataArray objects
gmsl_grids_orbiterror_2cm = xr.DataArray(ssh_weighted_mean_orbiterror, dims=['time'])
gmsl_grids_orbiterror_2cm = gmsl_grids_orbiterror_2cm.assign_coords({'time':grids.time})
gmsl_grids_orbiterror_2cm.name = 'gmsl_grids'
gmsl_grids_orbiterror_2cm

<xarray.DataArray 'gmsl_grids' (time: 923)> Size: 7kB
array([-3.36532325e-02, -3.21153236e-02, -2.93054564e-02, -3.04677060e-02,
       -3.13755450e-02, -3.28350760e-02, -3.16731694e-02, -3.31284392e-02,
       -3.28112035e-02, -3.89346454e-02, -4.02000457e-02, -4.03687840e-02,
       -4.09206988e-02, -4.23617291e-02, -4.33009912e-02, -4.02988641e-02,
       -4.00621715e-02, -3.82652121e-02, -4.08518422e-02, -3.85575776e-02,
       -3.85492005e-02, -3.75743524e-02, -3.85353786e-02, -3.81185336e-02,
       -3.73997635e-02, -3.68422122e-02, -3.75655812e-02, -3.69060495e-02,
       -3.92909664e-02, -3.78188697e-02, -3.66518817e-02, -3.74187968e-02,
       -3.46769315e-02, -3.36222827e-02, -3.24256845e-02, -3.21585456e-02,
       -3.02936157e-02, -2.90718554e-02, -2.75519243e-02, -2.81105651e-02,
       -2.89571089e-02, -3.04070140e-02, -3.31313677e-02, -3.13096235e-02,
       -2.81099821e-02, -2.72130169e-02, -3.14453731e-02, -3.19095960e-02,
       -3.03357176e-02, -3.43851359e-02, -3.82625921e-02, -3.91503755e-02,
       -3.54640487e-02, -3.30175379e-02, -3.21689409e-02, -3.34498294e-02,
       -3.23218837e-02, -3.34644366e-02, -3.33325157e-02, -3.62276214e-02,
       -3.62080546e-02, -3.55430395e-02, -3.86827612e-02, -3.92519133e-02,
       -4.02758854e-02, -3.89036151e-02, -3.71562639e-02, -3.74611417e-02,
       -3.58285642e-02, -3.58765026e-02, -3.23440167e-02, -3.18398836e-02,
       -2.97490081e-02, -3.01891161e-02, -2.73861527e-02, -2.76810502e-02,
       -2.54697382e-02, -2.66147734e-02, -2.79896403e-02, -2.81761290e-02,
...
        4.67449569e-02,  4.61202101e-02,  4.69677599e-02,  4.50997120e-02,
        4.35626700e-02,  4.03398479e-02,  3.81824195e-02,  4.21664161e-02,
        4.16986274e-02,  3.79399636e-02,  3.83707140e-02,  3.68080519e-02,
        3.78637123e-02,  3.52566520e-02,  3.89550592e-02,  3.66891284e-02,
        3.70746082e-02,  3.66133727e-02,  3.86766472e-02,  3.83351033e-02,
        3.64382057e-02,  3.82647309e-02,  3.74463798e-02,  3.68864454e-02,
        3.79288075e-02,  3.63096837e-02,  3.55315717e-02,  3.92125125e-02,
        3.90486472e-02,  4.05265236e-02,  4.21260660e-02,  4.32671648e-02,
        4.28503872e-02,  4.29316996e-02,  4.39818005e-02,  4.63375297e-02,
        4.58925631e-02,  4.84143187e-02,  4.82291043e-02,  4.92044840e-02,
        4.71234009e-02,  4.77100531e-02,  4.52867310e-02,  4.54700691e-02,
        4.24854997e-02,  3.86695336e-02,  3.70821516e-02,  3.67685572e-02,
        3.77260969e-02,  3.76726857e-02,  3.75652870e-02,  3.73016543e-02,
        3.93721040e-02,  4.01303843e-02,  4.13064640e-02,  4.13652926e-02,
        4.07831416e-02,  3.98460369e-02,  4.24977186e-02,  4.13981527e-02,
        4.17226959e-02,  4.04936768e-02,  4.10915067e-02,  4.27977264e-02,
        4.40981162e-02,  4.59504558e-02,  4.68706023e-02,  4.83521217e-02,
        4.91554160e-02,  4.76579516e-02,  4.79708421e-02,  4.98280355e-02,
        5.25416122e-02,  5.42105208e-02,  5.46847631e-02,  5.67741594e-02,
        5.53681175e-02,  5.59243016e-02,  5.34950304e-02])
Coordinates:
  * time     (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2017-12-25

### 3 errors: random noise + missing data + orbit error - 2cm

In [50]:
# After running Kevin's code simple gridder.py, grids are generating
ECCO_grids_files = list(grids_file_dir_3errors_2cm.glob('*SSHA_gridded*nc'))
ECCO_grids_files=np.sort(ECCO_grids_files)
print(ECCO_grids_files[0:5])
print('...')
print(ECCO_grids_files[-5:])

[PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_3errors_2cm_nosicapplied/SSHA_gridded_1992-09-27.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_3errors_2cm_nosicapplied/SSHA_gridded_1992-10-07.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_3errors_2cm_nosicapplied/SSHA_gridded_1992-10-17.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_3errors_2cm_nosicapplied/SSHA_gridded_1992-10-27.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_3errors_2cm_nosicapplied/SSHA_gridded_1992-11-06.nc')]
...
[PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_3errors_2cm_nosicapplied/SSHA_gridded_2017-11-25.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_3errors_2cm_nosicapplied/SSHA_gridded_2017-12-05.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_3errors_2cm_nosicapplied/SSHA_gridded_2017-12-15.nc')
 PosixPath('/home/jpluser/ECCO_GMSL/Data/gridding_output_3errors_2cm_nosicapplied/SSHA_gridded_2017-12-25.n

In [51]:
grids = xr.open_mfdataset(
        paths=ECCO_grids_files,
        combine='nested',
        concat_dim='time',
        decode_cf=True,
        compat='override',
        coords='minimal',
        chunks={'time': 1}  
    )   
grids

<xarray.Dataset> Size: 4GB
Dimensions:    (time: 924, latitude: 360, longitude: 720)
Coordinates:
  * latitude   (latitude) float64 3kB -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * longitude  (longitude) float64 6kB -179.8 -179.2 -178.8 ... 179.2 179.8
  * time       (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2018-01-04
Data variables:
    SSHA       (time, latitude, longitude) float64 2GB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float64 2GB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>

In [52]:
grids=grids.where(np.array(grids.counts)>=100)
grids=grids.sel(latitude=slice(-66,66)).sel(time=slice('1992-09-23','2017-12-31'))
grids

<xarray.Dataset> Size: 3GB
Dimensions:    (time: 923, latitude: 264, longitude: 720)
Coordinates:
  * latitude   (latitude) float64 2kB -65.75 -65.25 -64.75 ... 64.75 65.25 65.75
  * longitude  (longitude) float64 6kB -179.8 -179.2 -178.8 ... 179.2 179.8
  * time       (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2017-12-25
Data variables:
    SSHA       (time, latitude, longitude) float64 1GB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>
    counts     (time, latitude, longitude) float64 1GB dask.array<chunksize=(1, 264, 720), meta=np.ndarray>

In [53]:
[LO,LA]=np.meshgrid(grids.longitude,grids.latitude)
weights = np.cos(np.deg2rad(LA))
weights=np.tile(weights, [grids.SSHA.shape[0], 1, 1])
weights[np.where(~np.isfinite(grids.SSHA))]=np.nan
ssh_weighted_mean_3errors=np.nansum(grids.SSHA*weights,axis=(1,2))/np.nansum(weights,axis=(1,2))

In [54]:
# make new DataArray objects
gmsl_grids_3errors_2cm = xr.DataArray(ssh_weighted_mean_3errors, dims=['time'])
gmsl_grids_3errors_2cm = gmsl_grids_3errors_2cm.assign_coords({'time':grids.time})
gmsl_grids_3errors_2cm.name = 'gmsl_grids'
gmsl_grids_3errors_2cm

<xarray.DataArray 'gmsl_grids' (time: 923)> Size: 7kB
array([-3.35943869e-02, -3.20050689e-02, -3.06115412e-02, -3.00354192e-02,
       -3.16117846e-02, -3.38751699e-02, -3.19864942e-02, -3.29891413e-02,
       -3.24387294e-02, -3.86206565e-02, -4.00217541e-02, -3.96101546e-02,
       -4.09618098e-02, -4.16481541e-02, -4.25424602e-02, -4.00234933e-02,
       -3.99364968e-02, -3.76371218e-02, -4.11497953e-02, -3.81618013e-02,
       -3.79207416e-02, -3.70067792e-02, -3.88879024e-02, -3.94754996e-02,
       -3.75229006e-02, -3.64487698e-02, -3.81531676e-02, -3.74583393e-02,
       -3.94325031e-02, -3.76081175e-02, -3.58251502e-02, -3.71424614e-02,
       -3.45586189e-02, -3.36713637e-02, -3.22843738e-02, -3.22429266e-02,
       -2.96595861e-02, -2.79832002e-02, -2.75224118e-02, -2.82099439e-02,
       -3.01128215e-02, -3.03635763e-02, -3.29876704e-02, -3.11112471e-02,
       -2.78764367e-02, -2.66307695e-02, -3.13725436e-02, -3.21383428e-02,
       -3.01275577e-02, -3.36022474e-02, -3.81990222e-02, -3.93322169e-02,
       -3.53496285e-02, -3.26045241e-02, -3.21118367e-02, -3.33175782e-02,
       -3.17601369e-02, -3.35735881e-02, -3.29298323e-02, -3.68212503e-02,
       -3.59652585e-02, -3.54482681e-02, -3.84406259e-02, -3.93790244e-02,
       -4.08978681e-02, -3.87357355e-02, -3.81628273e-02, -3.73265318e-02,
       -3.60528035e-02, -3.65269435e-02, -3.16594135e-02, -3.18225364e-02,
       -2.99422813e-02, -2.97648049e-02, -2.69112784e-02, -2.80943494e-02,
       -2.57425328e-02, -2.59698033e-02, -2.88787625e-02, -2.80782114e-02,
...
        4.69170367e-02,  4.58790500e-02,  4.74059576e-02,  4.49589938e-02,
        4.32408928e-02,  3.98077492e-02,  3.83358972e-02,  4.16920502e-02,
        4.01406100e-02,  3.79251141e-02,  3.86298826e-02,  3.69761021e-02,
        3.80417905e-02,  3.57131667e-02,  3.86330735e-02,  3.61821560e-02,
        3.76238904e-02,  3.69132219e-02,  3.96291665e-02,  3.80314617e-02,
        3.68034903e-02,  3.81090945e-02,  3.70762462e-02,  3.64588182e-02,
        3.68037916e-02,  3.65299794e-02,  3.51655884e-02,  3.90758481e-02,
        3.89493567e-02,  3.98896197e-02,  4.19054032e-02,  4.34604129e-02,
        4.31559143e-02,  4.31991175e-02,  4.46168022e-02,  4.65999313e-02,
        4.63710052e-02,  4.79688774e-02,  4.72363130e-02,  4.88620205e-02,
        4.73194719e-02,  4.64708202e-02,  4.47340087e-02,  4.56665388e-02,
        4.18283360e-02,  3.90678421e-02,  3.65476287e-02,  3.81603888e-02,
        3.76392418e-02,  3.83195632e-02,  3.74241265e-02,  3.76580718e-02,
        3.96136476e-02,  4.08122172e-02,  4.15705941e-02,  4.11010137e-02,
        4.00683042e-02,  3.98136624e-02,  4.25871330e-02,  4.11427361e-02,
        4.16605938e-02,  4.00476125e-02,  4.07375428e-02,  4.29735287e-02,
        4.40405762e-02,  4.65805636e-02,  4.69734662e-02,  4.75920163e-02,
        4.95795640e-02,  4.70212834e-02,  4.84145531e-02,  4.99310172e-02,
        5.21996213e-02,  5.45448643e-02,  5.46464065e-02,  5.64560224e-02,
        5.52960142e-02,  5.57274424e-02,  5.34957102e-02])
Coordinates:
  * time     (time) datetime64[ns] 7kB 1992-09-27 1992-10-07 ... 2017-12-25

### Save to Disk

In [43]:
fname = output_dir / ('GMSL_grids_19922017_noerror_100points_nosicapplied.nc')
gmsl_grids.to_netcdf(fname)

In [44]:
fname = output_dir / ('GMSL_grids_19922017_randomnoise_100points_nosicapplied.nc')
gmsl_grids_randomnoise.to_netcdf(fname)

In [45]:
fname = output_dir / ('GMSL_grids_19922017_missingdata_100points_nosicapplied.nc')
gmsl_grids_missingdata.to_netcdf(fname)

In [46]:
fname = output_dir / ('GMSL_grids_19922017_orbiterror_2cm_100points_nosicapplied.nc')
gmsl_grids_orbiterror_2cm.to_netcdf(fname)

In [56]:
fname = output_dir / ('GMSL_grids_19922017_3errors_2cm_100points_nosicapplied.nc')
gmsl_grids_3errors_2cm.to_netcdf(fname)